In [12]:
import dart_fss as dart
import pymysql

In [13]:
import import_ipynb

In [14]:
# api 키 세팅
from settings import api_key

api_key = api_key
dart.set_api_key(api_key = api_key)

'\t1c6dbdbbb0cf307d3455b383d0b780d09e83bfb7'

In [25]:
from settings import conn

In [26]:
conn = conn

In [27]:
cursor = conn.cursor()

In [38]:
def create(sql):
    cursor.execute(sql)

In [29]:
# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()
sector_list = corp_list.sectors

Output()

Output()

Output()

# sector table create 

In [36]:
sql = "CREATE TABLE IF NOT EXISTS sector(name varchar(255) not null, primary key (name))"

In [39]:
# sector 테이블 create
create(sql)

In [40]:
conn.commit()

In [32]:
sql = "INSERT INTO sector (name) VALUES (%s)"

In [35]:
# sector 테이블 insert
for i in sector_list:
    cursor.execute(sql, (i))
    conn.commit()

IntegrityError: (1062, "Duplicate entry '1차 비철금속 제조업' for key 'sector.PRIMARY'")

# stock table create

In [28]:
sql = "CREATE TABLE IF NOT EXISTS stock(code varchar(9) not null, name varchar(255) not null, market varchar(1) not null, sector_name varchar(255) not null, primary key (code), foreign key (sector_name) references sector(name) on update restrict on delete restrict)"

In [29]:
# stock 테이블 create
create(sql)

# stock all record insert

In [30]:
sql = "INSERT INTO stock (code, name, market, sector_name) VALUES (%s, %s, %s, %s)"

In [31]:
# stock 테이블 insert
for i in sector_list:
    # i와 일대다 관계로 res_by_sector의 corp_code(DB X), corp_name(종목 이름), stock_code(주식 종목 코드)
    corps_by_y_sector = corp_list.find_by_sector(i, market ='Y') # 코스피
    corps_by_k_sector = corp_list.find_by_sector(i, market ='K') # 코스닥
    if corps_by_y_sector is not None:
        for j in corps_by_y_sector:
            try:
                cur.execute(sql, (j.stock_code, j.corp_name, 'Y', i)) 
            except:
                pass
    if corps_by_k_sector is not None:
        for j in corps_by_k_sector:
            try:
                cur.execute(sql, (j.stock_code, j.corp_name, 'K', i)) 
            except:
                pass

conn.commit()

In [ ]:
conn.close()

# ----------재무제표 반영하지 않음(즉 아래 줄부터)------------

# finance table create

In [24]:
sql = "CREATE TABLE IF NOT EXISTS finance(id int not null auto_increment, per decimal(3, 2) not null, pbr decimal(3, 2) not null, roe decimal(3, 2) not null, rate decimal(3, 2) not null, stock_id int not null, primary key (id), foreign key (stock_id) references stock(id) on update cascade on delete cascade)"

In [25]:
create(sql)

# finance all record insert

In [28]:
def get_roe(corp):
    try:
        fs = dart.fs.extract(corp_code = corp.corp_code, bgn_de = "20220929", report_tp = ['quarter'])
        df_fs = fs['bs']
        df_is = fs['is']
        
        # 당기순이익
        net_income_t = df_is.loc[df_is[('[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)', 'label_ko')] == '당기순이익(손실)']['20220701-20220930']
        net_income = int(net_income.iloc[0][0])
        
        # 자본총계
        sum_of_capital_t = df_fs.loc[df_fs[('[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW)', 'label_ko')] == '자본총계']['20220930']
        sum_of_capital = int(sum_of_capital_t.iloc[0][0])
        
        roe = (net_income/sum_of_capital)*100
        
        # 종목 pk 가져오기
        sql = "SELECT FROM stock where name='{0}' and code = '{1}'".format(corp.corp_name, corp.stock_code+".KS")
        with conn:
            with conn.cursor() as cur:
                cur.execute(sql)
                stock_id = cur.fetchall()[0][0]
            conn.commit()
        
        sql = "INSERT INTO finance (per, pbr, roe, rate, stock_id) VALUES (%d, %d, %d, %d, %d)"
        
        with conn:
            with conn.cursor() as cur:
                cur.execute(sql, (0, 0, roe, roe, stock_id))
            conn.commit()
    except:
        stock_delete(corp)

In [ ]:
for i in sector_list:
    corps_by_y_sector = corp_list.find_by_sector(i, market ='Y') # 코스피
    corps_by_k_sector = corp_list.find_by_sector(i, market ='K') # 코스닥
    
    for j in corps_by_y_sector:
        roe(j)
    
    for j in corps_by_k_sector:
        roe(j)

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Traceback (most recent call last):
  File "C:\Users\USER\Desktop\stock-recsys-pipeline\env\lib\site-packages\dart_fss\fs\extract.py", line 1411, in extract
    statements = analyze_report(report=report,
  File "C:\Users\USER\Desktop\stock-recsys-pipeline\env\lib\site-packages\dart_fss\fs\extract.py", line 1264, in analyze_report
    raise NotFoundConsolidated('Could not find consolidated financial statements')
dart_fss.errors.errors.NotFoundConsolidated: Could not find consolidated financial statements
C:\Users\USER\Desktop\stock-recsys-pipeline\env\lib\site-packages\dart_fss\fs\extract.py:1440: RuntimeWarning: Unable to extract financial statements: {'rcp_no': '20221114000694', 'corp_code': '00145464', 'corp_name': '이구산업', 'stock_code': '025820', 'corp_cls': 'Y', 'report_nm': '분기보고서 (2022.09)', 'flr_nm': '이구산업', 'rcept_dt': '20221114', 'rm': ''}.
  warnings.warn(warnings_text, RuntimeWarning)


...

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/1 [00:00<?, ?report/s]